In [44]:
import numpy as np
import torch
from torch import nn
from matplotlib import pyplot as plt
import re
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from collections import Counter
from typing import List, Tuple
import pandas as pd
from transformers import OpenAIGPTTokenizer, AutoTokenizer, AutoModel
import pickle
plt.rcParams['figure.figsize'] = 10, 6

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 2.1.1

In [46]:
"""
class BeowulfDataset(Dataset):
    def __init__(self, file_path: str, sequence_length: int, sequence_length_enc: int, picklefile=None):
        # Read the text file
        text = pandas.read_csv(file_path)
        text = text[["title", "content"]]
        text = text.dropna()
        # Remove non-alphanumeric characters and tokenize

        # Build vocabulary
        self.tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        
        # Create input-output pairs using sliding window
        self.sequence_length = sequence_length
        self.pairs = [(x_a[0], x_a[1][:10000]) for x_a in text.to_numpy()]
        if picklefile is None:
            for i in range(len(self.pairs)):
                if len(self.tokenizer(self.pairs[i][0],return_tensors="pt", truncation=True, max_length=self.sequence_length, padding="max_length")["input_ids"].squeeze()) == 0 or len(self.tokenizer(self.pairs[i][0],return_tensors="pt", truncation=True, max_length=self.sequence_length, padding="max_length")["input_ids"].squeeze()) == 0:
                    self.pairs.pop(i)
                i -= 1
            self.threes = []
            o = 0
            for pair in self.pairs:
                o+=1
                print("\r",o, end="")
                x, y = pair
                x = self.tokenizer(x, return_tensors="pt", truncation=True, max_length=sequence_length_enc, padding="max_length")["input_ids"].squeeze()
                y = self.tokenizer(y, return_tensors="pt", truncation=True, padding="max_length", max_length=self.sequence_length+1)["input_ids"].squeeze()
                if y is None or len(y) == 0:
                    continue
                for i in range(min(len(y), self.sequence_length+1)):
                    z = y[i]
                    y_z = y.clone()
                    y_z[i:] = self.tokenizer.pad_token_id
                    y_z = y_z[:self.sequence_length]
                    self.threes.append((x,y_z,z))
            with open("../threes.pickle", "wb") as f:
                pickle.dump(self.threes, f)
        else:
            with open(picklefile, "rb") as f:
                self.threes = pickle.load(f)

    def __len__(self):
        return len(self.threes)

    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        return self.threes[idx]

    def num_unique_tokens(self):
        return len(self.tokenizer)
    

    def tokens_to_text(self, tokens:torch.Tensor) -> str:

        if tokens.shape[0] == 1:
            tokens = [tokens.item()]
        else: 
            tokens = tokens.tolist()
        
        txt = [self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(token)) for token in tokens]
        return ' '.join(txt)
    
    def text_to_tokens(self, text: str) -> torch.Tensor:

        out = self.tokenizer(text)
        return out
"""

file_path = r'../gutenberg-poetry-dataset2.csv'
batch_size = 128
shuffle = True
sequence_length = 100  # for example, predicting 10th word based on previous 9 words
sequence_length_enc = 25

# dataset = BeowulfDataset(file_path, sequence_length, sequence_length_enc)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Example usage:
# print(f"Number of unique tokens in the dataset: {dataset.num_unique_tokens()}")

# print(dataset[128])
# uniques = dataset.num_unique_tokens()
# print(uniques)



In [47]:
text = pandas.read_csv(file_path)
text = text[["title", "content"]]
text = text.dropna()

In [72]:
def remove_volume(text):
    pattern = r'— volume \d+'
    cleaned_text = re.sub(pattern, '', text)

    pattern = r'.? vol. \d+'
    cleaned_text = re.sub(pattern, '', cleaned_text)

    pattern = r',? vol. \d+'
    cleaned_text = re.sub(pattern, '', cleaned_text)

    pattern = r',? volume \d+'
    cleaned_text = re.sub(pattern, '', cleaned_text)

    return cleaned_text

def remove_after_tab(text):
    return text.split('\t')[0]

def to_lower(text):
    return text.lower()

def remove_part(text):
    pattern = r'— part \d+'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# pipeline
def data_pipeline(transformations):

    def pipeline(text):
        output = text[:]
        for transformation in transformations:
            output = transformation(output)
    
        return output

    return pipeline

def remove_version(text):
    pattern =  r',? version \d+(\.\d+)+'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Funkcja do usuwania ", 24 jul 1996"
def remove_date(text):
    pattern = r',? [a-zA-Z]+ \d{1,2},? \d{4}'
    cleaned_text = re.sub(pattern, '', text)

    pattern = r',? \d{4}'
    cleaned_text = re.sub(pattern, '', cleaned_text)
    
    return cleaned_text

def remove_after_last_comma(text):
    return ', '.join(text.split(',')[:-1])



transformations = [to_lower,
                  remove_volume,
                  remove_after_tab,
                  remove_part,
                  remove_version,
                  remove_date,
                  ]

pipeline = data_pipeline(transformations)

new_titles = text['title'].apply(pipeline)


for i in range(len(new_titles)):
    print(new_titles.iloc[i])

paradise lost
aesop's fables
far from the madding crowd
the communist manifesto
violists
the mayor of casterbridge
the island of doctor moreau
charlotte temple
child christopher and goldilind the fair
clotelle; or, the colored heroine, a tale of the southern states; or, the president's daughter
my antonia
ballads of a cheechako
a dome of many-coloured glass
trees and other poems
records of a family of engineers
the golden age
beauty and the beast, and tales of home
paul prescott's charge
the early short fiction of edith wharton 
the cash boy
the flirt
the market-place
ballad of reading gaol
indian boyhood
the soul of the indian
dracula
hesiod, the homeric hymns, and homerica
the harvester
of human bondage
in flanders fields and other poems
good stories for holidays
the mad king
the lost prince
the great god pan
tales and fantasies
the great war syndicate
frivolous cupid
the conflict
the master key, an electrical fairy tale founded upon the mysteries of electricity
the crowd; study of t